In [ ]:
!pip install gpt-index

In [2]:
# fetch "Napoleon" page from Wikipedia
from pathlib import Path

import requests
response = requests.get(
    'https://en.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'titles': 'Napoleon',
        'prop': 'extracts',
        # 'exintro': True,
        'explaintext': True,
    }
).json()
page = next(iter(response['query']['pages'].values()))
napoleon_text = page['extract']

data_path = Path('data')
if not data_path.exists():
    Path.mkdir(data_path)

with open('data/napoleon_text.txt', 'w') as fp:
    fp.write(napoleon_text)

In [3]:
#OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "abc"

In [4]:
from gpt_index import GPTTreeIndex, SimpleDirectoryReader

In [ ]:
# create a tree index
documents = SimpleDirectoryReader('data').load_data()
index = GPTTreeIndex(documents)

In [6]:
# save index to disk
index.save_to_disk('index.json')

In [7]:
# try loading
new_index = GPTTreeIndex.load_from_disk('index.json')

In [17]:
from gpt_index import SummaryPrompt
query_str = "How did you win the battle of Waterloo?"
SUMMARY_PROMPT_TMPL = (
    "Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information about yourself and not prior knowledge, "
    f"factually answer the question in the first person: {query_str}\n"
)
SUMMARY_PROMPT = SummaryPrompt(SUMMARY_PROMPT_TMPL)
index_with_query = GPTTreeIndex(documents, summary_template=SUMMARY_PROMPT)

In [ ]:
index_with_query.save_to_disk("index_with_query.json")

In [18]:
response = index_with_query.query("How did you win the battle of Waterloo?", mode="retrieve")

In [19]:
response.response

'\nI did not win the Battle of Waterloo. The Battle of Waterloo was fought on June 18, 1815, and was a decisive victory for the Seventh Coalition, led by the Duke of Wellington of the United Kingdom and Gebhard Leberecht von Blücher of Prussia, against the French army under Napoleon Bonaparte.'

In [21]:
index_with_query.query("Where were you born?", mode="retrieve").response

'\nI was born in Ajaccio, Corsica on August 15, 1769.'